In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
import pandas as pd

In [11]:
# Load the dataset
data = pd.read_csv('/Users/grace/Desktop/Courses/Master_thesis/dataset1.csv')

In [12]:
# Separate the target variable
target_variable = 'time_on_site [Minutes]'
X = data.drop(target_variable, axis=1)
y = data[target_variable]

In [28]:
'''from sklearn.model_selection import StratifiedKFold

# Initialize the StratifiedKFold iterator
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize a list to store the accuracy scores
accuracy_scores = []

# Loop through each training/testing split
for train_index, test_index in skf.split(X, y):
    # Split the data into training and testing sets for this iteration
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Fit the classifier on the training data
    best_rf.fit(X_train, y_train)
    # Make predictions on the test data
    y_pred = best_rf.predict(X_test)
    
    # Calculate the accuracy score for this iteration
    accuracy = accuracy_score(y_test, y_pred)
    
    # Add the accuracy score to the list
    accuracy_scores.append(accuracy)

# Calculate the mean accuracy score across all iterations
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)

print("Mean accuracy score: {:.2f}".format(mean_accuracy))'''

C:\Users\grace\anaconda3\envs\XAI\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


Mean accuracy score: 0.80


In [13]:
# Initialize the Random Forest classifier
rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

In [14]:
# Initialize the LOOCV iterator
loo = LeaveOneOut()

# Initialize a list to store the mean squared error scores
mse_scores = []

In [15]:
# Loop through each training/testing split
for train_index, test_index in loo.split(X):
    # Split the data into training and testing sets for this iteration
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Fit the classifier on the training data
    rf.fit(X_train, y_train)
    # Make predictions on the test data
    y_pred = rf.predict(X_test)
    
    # Calculate the mean squared error score for this iteration
    mse = mean_squared_error(y_test, y_pred)
    
    # Add the mean squared error score to the list
    mse_scores.append(mse)


In [17]:
# Calculate the mean mean squared error score across all iterations
mean_mse = sum(mse_scores) / len(mse_scores)

print("Mean mean squared error score: {:.2f}".format(mean_mse))

Mean mean squared error score: 509.71


In [18]:
# Get the feature importances
importances = rf.feature_importances_

# Create a DataFrame of feature importances sorted from highest to lowest
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances}).sort_values('importance', ascending=False)

# Print the top 10 features with their importances
print(feature_importances.head(10))

            feature  importance
2       value [USD]    0.487990
0       customer_id    0.277989
3    clicks_in_site    0.137192
1  product_category    0.096829


In [19]:
# Select the top k features
k = 10
top_k_features = feature_importances['feature'][:k].tolist()

# Select the features from the dataset
X_train_selected = X_train[top_k_features]
X_test_selected = X_test[top_k_features]

# Train the Random Forest regressor on the selected features
rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rf.fit(X_train_selected, y_train)

RandomForestRegressor(max_depth=5, random_state=42)

In [20]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [21]:
# Define the hyperparameter grid to search
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}


In [37]:

# Create a RandomizedSearchCV object with the Random Forest classifier and the hyperparameter grid
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the RandomizedSearchCV object on the training data
rf_random.fit(X_train, y_train)

# Print the best hyperparameters found by the search
print(rf_random.best_params_)

# Use the best hyperparameters to fit a new Random Forest model on the training data
best_rf = RandomForestRegressor(n_estimators=rf_random.best_params_['n_estimators'],
                                  max_features=rf_random.best_params_['max_features'],
                                  max_depth=rf_random.best_params_['max_depth'],
                                  min_samples_split=rf_random.best_params_['min_samples_split'],
                                  min_samples_leaf=rf_random.best_params_['min_samples_leaf'],
                                  bootstrap=rf_random.best_params_['bootstrap'],
                                  random_state=42)

best_rf.fit(X_train, y_train)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': True}


RandomForestRegressor(max_depth=90, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=10, n_estimators=200, random_state=42)

In [38]:
# Make predictions on the test data using the tuned model
y_pred = best_rf.predict(X_test)
print(y_pred)

[28.64059465]


In [39]:
# Calculate the mean squared error for the tuned model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error of tuned model: {:.2f}".format(mse))

Mean Squared Error of tuned model: 300.70


# evaluating the performance of the model using a combination of accuracy and AUC-ROC metrics.


In [40]:
from sklearn.metrics import r2_score

In [41]:
# Calculate the R-squared score for the tuned model
r2 = r2_score(y_test, y_pred)
print("R-squared score of tuned model: {:.2f}".format(r2))

R-squared score of tuned model: nan


C:\Users\grace\anaconda3\envs\XAI\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [36]:
print(np.unique(y_test))

[11.3]
